In [1]:
# replace the standard sqlite3 module with pysqlite3
# for compatibility with Chroma
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


import langchain
import os
import bs4
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# load environment variables
load_dotenv()

True

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# credential json not required if you are working within vertex AI
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/LLM-agent-with-Gemini/fleet-anagram-244304-7dafcc771b2f.json"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") # only if you are using text embedding model from google

In [3]:
search = TavilySearchResults(max_results=3)
result = search.invoke("What is Julia language known for?")

# Formatting
for i, item in enumerate(result, start=1):
    print(f"{i}. URL: {item['url']}\nContent: {item['content']}\n")

1. URL: https://www.datacamp.com/blog/what-is-julia-used-for
Content: The Julia website lists many of the language's applications, and we've highlighted some of the key ones below: 6. Pharmaceutical Modelling and Development. Julia is widely used in the medical field. Researchers use Julia to identify and test new drugs, optimize treatments, and for diagnosis and treatments.

2. URL: https://en.wikipedia.org/wiki/Julia_(programming_language)
Content: Julia is a high-level, general-purpose dynamic programming language, most commonly used for numerical analysis and computational science. Distinctive aspects of Julia's design include a type system with parametric polymorphism and the use of multiple dispatch as a core programming paradigm, efficient garbage collection, and a just-in-time (JIT) compiler (with support for ahead-of-time ...

3. URL: https://julialang.org/
Content: Julia has been downloaded over 45 million times and the Julia community has registered over 10,000 Julia package

In [4]:
web_paths = [
    "https://google.github.io/styleguide/pyguide.html",
    "https://google.github.io/styleguide/Rguide.html",
    # "https://google.github.io/styleguide/cppguide.html",
    # "https://google.github.io/styleguide/go/",
    # "https://google.github.io/styleguide/javaguide.html",
    # "https://google.github.io/styleguide/jsguide.html",
    # "https://google.github.io/styleguide/tsguide.html"
]

# Only keep post title, headers, and content from the full HTML.
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

docs = []
for path in web_paths:
    loader = WebBaseLoader(web_paths=(path,),
                        #    bs_kwargs={"parse_only": bs4_strainer}
    )
    docs += loader.load()

In [5]:
len(docs[0].page_content)

88189

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

124

In [ ]:
all_splits[-1].metadata

In [7]:
vectorstore = FAISS.from_documents(documents=all_splits, embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

/home/codespace/.cache/pypoetry/virtualenvs/llm-agent-with-gemini-6S2l9oJE-py3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/codespace/.cache/pypoetry/virtualenvs/llm-agent-with-gemini-6S2l9oJE-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
retriever_tool = create_retriever_tool(
    retriever,
    "programming_style_guide_search",
    "Reference for information about styling rules for programming languages",
)

In [9]:
tools = [search, retriever_tool]

In [35]:
llm = ChatVertexAI(model="gemini-1.5-flash")

In [36]:
agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

In [ ]:
import json

def print_formatted_response(response):
    for i in range(4):
        if i == 0:
            print(f"Question: {response['messages'][0].content}")
        elif i == 1:
            function_call = response['messages'][i].additional_kwargs.get('function_call', {})
            name = function_call.get('name', '')
            print(f"Tool: {name}")
        elif i == 2:
            data = json.loads(response['messages'][2].content)
            print("Citation:")
            for item in data:
                print(item['url'])
        else:
            print(f"Answer: {response['messages'][3].content}")
        print()

In [37]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What are the differences between langchain and llamaindex, search if it's not in retriever")]}
)
response["messages"]

[HumanMessage(content="What are the differences between langchain and llamaindex, search if it's not in retriever", id='8dc0fb80-6e77-43b4-af88-1874425df09c'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'programming_style_guide_search', 'arguments': '{"query": "differences between langchain and llamaindex"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 103, 'candidates_token_count': 16, 'total_token_count': 119}}, id='run-e1977607-44d7-406a-b345-fa420a9f1847-0', tool_calls=[{'name': 'programming_style_guide_search',

In [13]:
print(response["messages"][-1].content)

LlamaIndex and LangChain are both frameworks designed to simplify the development of applications using large language models (LLMs). Here are some key differences:

* **Purpose and Scope:**
    * **LangChain** is a more multipurpose framework, suitable for building a wider range of applications, including chatbots, data analysis tools, and more. It focuses on providing flexibility and extensive customization options.
    * **LlamaIndex** is more specialized and focuses on facilitating interactions between LLMs and external data. It excels in applications that require querying, summarizing, or extracting information from various data sources.

* **Features:**
    * **LangChain** offers a broader selection of integrations with external tools and APIs, making it more versatile. It provides features for prompt management, chain building (combining multiple LLM calls), and agent systems. 
    * **LlamaIndex** shines in its data indexing and querying capabilities. It offers various data con

In [38]:
# streamin
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is Vajra")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is Vajra"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 87, 'candidates_token_count': 13, 'total_token_count': 100}}, id='run-3c2e77a0-259e-4ec0-a7ab-ebeaad9402ac-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is Vajra'}, 'id': '1f586c99-46f4-4a3b-b1a1-5cc247a80633'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.britannica.com/to

In [39]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [40]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, checkpointer=memory
)

config = {"configurable": {"thread_id": "abc123"}}

In [41]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi, I'm Hanyu")]}, config
):
    print(chunk)
    print()

{'agent': {'messages': [AIMessage(content='Nice to meet you, Hanyu! 👋 What can I do for you today? 😊 \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 90, 'candidates_token_count': 20, 'total_token_count': 110}}, id='run-a64ff6c5-c971-4ddf-b0b5-34044e1523c2-0')]}}



In [42]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What's my name?")]}, config
):
    print(chunk)
    print()

{'agent': {'messages': [AIMessage(content='Your name is Hanyu! 😊  I remember that from our first interaction.  Is there anything else I can help you with? \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 116, 'candidates_token_count': 29, 'total_token_count': 145}}, id='run-406fb25b-6f74-4e7d-8676-f8376bb26c1b-0')]}}



In [44]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Suggest some use cases for LLM Agent")]}, config
)

print(response["messages"][-1].content)
    

You're getting into the exciting world of LLM agents! Here are some use cases for them, ranging from practical to innovative:

**Practical Use Cases:**

* **Customer Service Chatbots:** LLMs can handle initial customer inquiries, provide product information, and even resolve simple issues. This frees up human agents for more complex tasks.
* **Personal Assistants:** An LLM agent could manage your calendar, send reminders, book appointments, and even make recommendations based on your preferences.
* **Content Creation:**  LLM agents can assist with writing articles, blog posts, social media content, and even code, significantly speeding up the process and potentially improving quality.
* **Research Assistants:** These agents could analyze research papers, identify relevant sources, and summarize findings, making research more efficient.
* **Data Analysis:** LLMs can analyze large datasets, identify patterns, and generate reports, empowering data-driven decision-making.

**Innovative Use